# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры(не менее 0.59).
Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [28]:
data = pd.read_csv('Churn.csv')

In [29]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


**Преобразуем категориальные признаки в числовые техникой прямого кодирования.**

In [31]:
data_drop = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data_ohe = pd.get_dummies(data_drop, drop_first=True)
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1.0,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8.0,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1.0,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2.0,125510.82,1,1,1,79084.10,0,False,True,False


**Столбцы RowNumber, CustomerId, Surname были удалены, так как они не несут полезной информации для модели.**

In [32]:
data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             9091 non-null   float64
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_Germany  10000 non-null  bool   
 10  Geography_Spain    10000 non-null  bool   
 11  Gender_Male        10000 non-null  bool   
dtypes: bool(3), float64(3), int64(6)
memory usage: 732.5 KB


In [33]:
data_ohe['Tenure'].value_counts(dropna=False)

Tenure
1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
NaN     909
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: count, dtype: int64

**В столбце Tenure обнаружены пропуски (909 значений).**

In [34]:
data_ohe[data_ohe['Tenure'].isna()].head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
30,591,39,NaN,0.00,3,1,0,140469.38,1,False,True,False
48,550,38,NaN,103391.38,1,0,1,90878.13,0,True,False,True
51,585,36,NaN,146050.97,2,0,0,86424.57,0,True,False,True
53,655,41,NaN,125561.97,1,0,0,164040.94,1,True,False,True
60,742,35,NaN,136857.00,1,0,0,84509.57,0,True,False,True


In [35]:
data_ohe[data_ohe['Tenure'] == 0.0]['Exited'].mean()

np.float64(0.2356020942408377)

In [36]:
data_ohe[data_ohe['Tenure'].isna()]['Exited'].mean()

np.float64(0.20132013201320131)

In [37]:
data_ohe[data_ohe['Tenure'] == data_ohe['Tenure'].median()]['Exited'].mean()

np.float64(0.2017259978425027)

**Заполним пропуски медианным значением.**

In [38]:
data_ohe['Tenure'] = data_ohe['Tenure'].fillna(data_ohe['Tenure'].median())

In [39]:
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1.0,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8.0,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1.0,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2.0,125510.82,1,1,1,79084.10,0,False,True,False


In [45]:
data_ohe['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

**Целевая переменная Exited имеет дисбаланс: класс 0 (не ушедшие клиенты) — 7963, класс 1 (ушедшие клиенты) — 2037.**

## Исследование задачи

In [40]:
target = data_ohe['Exited']

In [41]:
features = data_ohe.drop('Exited', axis=1)

In [62]:
# Первое разделение: тестовая выборка (20%)
features_valid, features_test, target_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

# Второе разделение: обучающая (60%) и валидационная (20%)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.25, random_state=42
)

# Итоговые размеры:
print(f"Обучающая: {features_train.shape[0]} строк")
print(f"Валидационная: {features_valid.shape[0]} строк")
print(f"Тестовая: {features_test.shape[0]} строк")

Обучающая: 6000 строк
Валидационная: 2000 строк
Тестовая: 2000 строк


In [63]:
best_f1 = 0
for est in [10, 50, 100, 200]:  # Увеличили диапазон
    for depth in [5, 10, None]:  # None — без ограничения глубины
        model = RandomForestClassifier(
            n_estimators=est,
            max_depth=depth,
            random_state=42
        ) # обучите модель с заданным количеством деревьев
        model.fit(features_train, target_train) # обучите модель на тренировочной выборке
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)# посчитайте качество модели на валидационной выборке
        if f1 > best_f1:
            best_f1 = f1
            n_estimators = est
            max_depth = depth
print("n_estimators =", n_estimators,"max_depth =", max_depth,", f1 ", best_f1)


n_estimators = 100 max_depth = None , f1  0.6017964071856288


**Для модели RandomForestClassifier был выполнен перебор гиперпараметров (n_estimators и max_depth). Лучший результат (F1 = 0.602) достигнут при n_estimators=100 и max_depth=None (без ограничения глубины деревьев).**

In [64]:
model = RandomForestClassifier(
            n_estimators=100,
            random_state=42
        ) # обучите модель с заданным количеством деревьев
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
roc_auc = roc_auc_score(target_valid, predicted_valid)
print(f1, roc_auc)

0.6017964071856288 0.7263281718441759


In [65]:
pd.Series(predicted_valid).value_counts()

0    1745
1     255
Name: count, dtype: int64

## Борьба с дисбалансом

In [66]:
# увеличение выборки редкого класса UPSAMPLING
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled,
                                                   target_upsampled,
                                                   random_state=42)
    return features_upsampled, target_upsampled

In [70]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 7)

In [71]:
model = RandomForestClassifier(
            n_estimators=100,
            random_state=42
        ) # обучите модель с заданным количеством деревьев
model.fit(features_upsampled, target_upsampled) # обучите модель на тренировочной выборке
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
roc_auc = roc_auc_score(target_valid, predicted_valid)
print(f1, roc_auc)

0.6189856957087126 0.750958529578247


**Для устранения дисбаланса применен метод увеличения выборки редкого класса (класс 1). Количество объектов класса 1 было увеличено в 7 раз.**
После upsampling модель показывает улучшенные результаты:

F1-мера: 0.619 (на валидационной выборке).

AUC-ROC: 0.751, что указывает на хорошую способность модели разделять классы.

## Тестирование модели

In [72]:
predicted_test = model.predict(features_test)
f1 = f1_score(target_test, predicted_test)
roc_auc = roc_auc_score(target_test, predicted_test)
print(f1, roc_auc)

0.6131191432396251 0.752456254522596


**Результаты на тестовой выборке:**

**F1-мера:** 0.613.

**AUC-ROC:** 0.752.

**Интерпретация:**
- F1-мера выше требуемого порога (0.59), что означает успешное выполнение задачи.
- AUC-ROC близок к 0.75, что говорит о средней, но достаточной для данной задачи, предсказательной способности модели.
- Модель лучше предсказывает класс 0 (не ушедшие клиенты), но upsampling помог улучшить предсказания для класса 1.